# Task 4

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
#Reading the dataset
#i have marged the part 1 and part 2 of the HAM10000_images.
root = 'HAM10000_images'
image_filenames=[]
images = []

for file_name in os.listdir('./HAM10000_images'):
    img = plt.imread(os.path.join(root, file_name))
    images.append(img.flatten())  # Flatten image into a 1D array
    image_filenames.append(file_name.split('.')[0])


In [ ]:
import random

# Display a random sample of images
sample_size = 2  # You can change this to display a different number of images

# Randomly select sample images
sample_images = random.sample(image_filenames, sample_size)

# Loop through and display the sample images
for filename in sample_images:
    file_path = os.path.join(root,file_name)
    image = Image.open(file_path)
    plt.imshow(image)
    plt.title(filename)
    plt.axis('off')
    plt.show()

In [ ]:
image_filenames

In [ ]:
#total samples
len(image_filenames)

In [ ]:
df=pd.read_csv('HAM10000_metadata.csv')

In [ ]:
fnames = df['image_id']
labels_unordered = df['dx']

In [ ]:
df.head()

In [ ]:
labels = [None]*(len(labels_unordered))

In [ ]:

for i in range(0,len(image_filenames)):
    
    for j in range(0,len(fnames)):
        
        if image_filenames[i] == fnames[j]:
            labels[i] = labels_unordered[j]
        

In [ ]:
labels

In [ ]:
#Labels
y[:10]

In [ ]:
#input images are flattened into 1D array.
x[:10]

In [ ]:
# Sample data with string labels
labels
# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the encoder on your labels and transform the labels into numerical values
numeric_labels = label_encoder.fit_transform(labels)

# Display the mapping between original labels and numerical values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

# Print the converted numerical labels
print("Numeric Labels:", numeric_labels)

In [ ]:
#This is the x and y calculated the length of both.
x = images   #these are the input images.
y = numeric_labels #these are the labels against images.
len(x),len(y)

# Train_test Data_Split

In [ ]:
#Setting up the random state
np.random.seed(42)

# Splitting the data into initial training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2)

print(f"This is the initial training data division: ({len(X_train)}, {len(Y_train)})")
print(f"This is the initial test data division: ({len(X_test)}, {len(Y_test)})")

# Now, let's create a training subset from the initial training data
train_subset_size = 0.6  # Adjust this value as needed
X_train_subset, _, Y_train_subset, _ = train_test_split(X_train, Y_train, train_size=train_subset_size, random_state=42)

print(f"This is the training subset division: ({len(X_train_subset)}, {len(Y_train_subset)})")

# Convert to NumPy arrays

X_train_subset = np.array(X_train_subset)
Y_train_subset = np.array(Y_train_subset)


# Fitting the Model on Data

In [ ]:
#Shifting my String data of labels into the numeric form

In [ ]:
# Create MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_subset.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')  # 15 output classes for 15 subjects
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_subset, Y_train_subset, epochs=15, batch_size=32, validation_split=0.2)

train_loss, train_accuracy = model.evaluate(X_train_subset, Y_train_subset)

print(f'Training accuracy: {train_accuracy * 100:.2f}%')


# Testing the Model

In [ ]:
# Test the model
X_test_Shape = np.array(X_test)
y_pred = model.predict(X_test_Shape)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, and f1-score
precision = precision_score(Y_test, y_pred_classes, average='macro')
recall = recall_score(Y_test, y_pred_classes, average='macro')
f1 = f1_score(Y_test, y_pred_classes, average='macro')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')